In [8]:
########### Python 3.2 #############
import requests
import json
import pandas as pd

############

In [45]:
url = "https://api.energyville.be/smarthordc/v1/"
source = 'Weather.Forecast.Rebase'
params = {
    'start': '2023-01-01T00:00:00Z',
    'end': '2023-01-01T02:00:00Z',
    'keys': ['Temperature']#, 'CloudCover', 'SolarDownwardRadiation', 'SurfaceDownSolarDiffuseRadiation', 'SurfaceDownSolarDirectRadiation', 'WindSpeed', 'RelativeHumidity']
}
body = {
    "Location": ["EnergyVille 1, BE"]
}
headers = {
    'Ocp-Apim-Subscription-Key': '1e3353b6342142cd96b61451416bd9cb',
}
response = requests.post(f'{url}/api/sources/{source}/timeseries?', headers=headers, params=params, json=body)
df = pd.DataFrame(json.loads(response.text))
########################

In [49]:
df.tail(20)

,Timestamp,TimeZoneOffset,MeasurementId,Value,Unit,Key,ForecastGeneratedTimestamp,ForecastWindow,ForecastStart,Latitude,Longitude,MetaData,Location
124,2023-01-01T01:00:00Z,,Weather.Forecast.Rebase.50994716-05539433.Temp...,13.70,degC,Temperature,2023-01-01 11:00:00+00:00,2.00:00:00,2023-01-01T00:00:00Z,50.994716,5.539433,"{'latitude': 50.994716, 'longitude': 5.539433}","EnergyVille 1, BE"
125,2023-01-01T01:00:00Z,,Weather.Forecast.Rebase.50994716-05539433.Temp...,12.39,degC,Temperature,2022-12-31 04:00:00+00:00,2.00:00:00,2022-12-31T00:00:00Z,50.994716,5.539433,"{'latitude': 50.994716, 'longitude': 5.539433}","EnergyVille 1, BE"
126,2023-01-01T01:00:00Z,,Weather.Forecast.Rebase.50994716-05539433.Temp...,13.70,degC,Temperature,2023-01-01 22:00:00+00:00,2.00:00:00,2023-01-01T00:00:00Z,50.994716,5.539433,"{'latitude': 50.994716, 'longitude': 5.539433}","EnergyVille 1, BE"
127,2023-01-01T01:00:00Z,,Weather.Forecast.Rebase.50994716-05539433.Temp...,12.39,degC,Temperature,2022-12-31 02:00:00+00:00,2.00:00:00,2022-12-31T00:00:00Z,50.994716,5.539433,"{'latitude': 50.994716, 'longitude': 5.539433}","EnergyVille 1, BE"
128,2023-01-01T01:00:00Z,,Weather.Forecast.Rebase.50994716-05539433.Temp...,12.39,degC,Temperature,2022-12-30 23:00:00+00:00,2.00:00:00,2022-12-30T00:00:00Z,50.994716,5.539433,"{'latitude': 50.994716, 'longitude': 5.539433}","EnergyVille 1, BE"
129,2023-01-01T01:00:00Z,,Weather.Forecast.Rebase.50994716-05539433.Temp...,13.70,degC,Temperature,2023-01-01 17:00:00+00:00,2.00:00:00,2023-01-01T00:00:00Z,50.994716,5.539433,"{'latitude': 50.994716, 'longitude': 5.539433}","EnergyVille 1, BE"
130,2023-01-01T01:00:00Z,,Weather.Forecast.Rebase.50994716-05539433.Temp...,13.70,degC,Temperature,2023-01-01 01:00:00+00:00,2.00:00:00,2023-01-01T00:00:00Z,50.994716,5.539433,"{'latitude': 50.994716, 'longitude': 5.539433}","EnergyVille 1, BE"
131,2023-01-01T01:00:00Z,,Weather.Forecast.Rebase.50994716-05539433.Temp...,12.36,degC,Temperature,2022-12-31 14:00:00+00:00,2.00:00:00,2022-12-31T00:00:00Z,50.994716,5.539433,"{'latitude': 50.994716, 'longitude': 5.539433}","EnergyVille 1, BE"
132,2023-01-01T01:00:00Z,,Weather.Forecast.Rebase.50994716-05539433.Temp...,14.58,degC,Temperature,2023-01-01 10:00:00+00:00,2.00:00:00,2023-01-01T00:00:00Z,50.994716,5.539433,"{'latitude': 50.994716, 'longitude': 5.539433}","EnergyVille 1, BE"
133,2023-01-01T01:00:00Z,,Weather.Forecast.Rebase.50994716-05539433.Temp...,12.30,degC,Temperature,2022-12-31 09:00:00+00:00,2.00:00:00,2022-12-31T00:00:00Z,50.994716,5.539433,"{'latitude': 50.994716, 'longitude': 5.539433}","EnergyVille 1, BE"


In [65]:
#Only use the forecast at 11 o clock
# Convert ForecastGeneratedTimestamp to datetime if it's not already in datetime format
df['ForecastGeneratedTimestamp'] = pd.to_datetime(df['ForecastGeneratedTimestamp'])
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Round the ForecastGeneratedTimestamp to the nearest hour
df['ForecastGeneratedTimestamp'] = df['ForecastGeneratedTimestamp'].dt.round('H')

# Select rows with '11:00:00' in RoundedForecastGeneratedTimestamp
DA_forecast_datetimes = pd.to_datetime(df['Timestamp'].dt.date - pd.Timedelta(days=1)) + pd.Timedelta(hours=11)
DA_forecast_datetimes.tz_localize('UTC')
print(DA_forecast_datetimes)
print(df['ForecastGeneratedTimestamp'])
DA_forecasts = df[df['ForecastGeneratedTimestamp'] == (pd.to_datetime(df['Timestamp'].dt.date - pd.Timedelta(days=1)) + pd.Timedelta(hours=11)).tz_localize('UTC')]

# Drop unnecessary columns
#closest_forecasts = closest_forecasts.drop('RoundedForecastGeneratedTimestamp', axis=1)

# Display or use the resulting DataFrame (closest_forecasts)
DA_forecasts.head()


0     2022-12-31 11:00:00
1     2022-12-31 11:00:00
2     2022-12-31 11:00:00
3     2022-12-31 11:00:00
4     2022-12-31 11:00:00
              ...        
139   2022-12-31 11:00:00
140   2022-12-31 11:00:00
141   2022-12-31 11:00:00
142   2022-12-31 11:00:00
143   2022-12-31 11:00:00
Name: Timestamp, Length: 144, dtype: datetime64[ns]
0     2022-12-30 04:00:00+00:00
1     2023-01-01 15:00:00+00:00
2     2022-12-31 22:00:00+00:00
3     2022-12-31 01:00:00+00:00
4     2022-12-31 10:00:00+00:00
                 ...           
139   2023-01-01 20:00:00+00:00
140   2023-01-01 12:00:00+00:00
141   2022-12-30 20:00:00+00:00
142   2023-01-01 05:00:00+00:00
143   2023-01-01 14:00:00+00:00
Name: ForecastGeneratedTimestamp, Length: 144, dtype: datetime64[ns, UTC]


TypeError: index is not a valid DatetimeIndex or PeriodIndex